In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [2]:
col_names = [f'A{i}' for i in range(1, 17)]
df = pd.read_csv('crx.data', names=col_names, header=0)

In [3]:
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
1,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
2,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
3,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
4,b,32.08,4.000,u,g,m,v,2.50,t,f,0,t,g,00360,0,+


In [4]:
df.describe()

,A3,A8,A11,A15
count,689.000000,689.000000,689.000000,689.000000
mean,4.765631,2.224819,2.402032,1018.862119
std,4.978470,3.348739,4.866180,5213.743149
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.165000,0.000000,0.000000
50%,2.750000,1.000000,0.000000,5.000000
75%,7.250000,2.625000,3.000000,396.000000
max,28.000000,28.500000,67.000000,100000.000000


In [5]:
df = df.replace('?', np.NaN)

In [6]:
df.isnull().sum()

A1     12
A2     12
A3      0
A4      6
A5      6
A6      9
A7      9
A8      0
A9      0
A10     0
A11     0
A12     0
A13     0
A14    13
A15     0
A16     0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689 entries, 0 to 688
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      677 non-null    object 
 1   A2      677 non-null    object 
 2   A3      689 non-null    float64
 3   A4      683 non-null    object 
 4   A5      683 non-null    object 
 5   A6      680 non-null    object 
 6   A7      680 non-null    object 
 7   A8      689 non-null    float64
 8   A9      689 non-null    object 
 9   A10     689 non-null    object 
 10  A11     689 non-null    int64  
 11  A12     689 non-null    object 
 12  A13     689 non-null    object 
 13  A14     676 non-null    object 
 14  A15     689 non-null    int64  
 15  A16     689 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.2+ KB


In [8]:
df = df.replace({'A1': {'a': 1, 'b': 0}, 'A9': {'t': 1, 'f': 1}, 'A10': {'t': 1, 'f': 1}, 'A12': {'t': 1, 'f': 1}, 'A16': {'+': 1, '-': 0}})

In [9]:
df = df.astype({'A1': 'boolean', 'A2': 'float64', 'A3': 'float64', 'A8': 'float64', 'A9': 'boolean', 'A10': 'boolean', 'A11': 'Int64', 'A12': 'boolean', 'A14': 'Int64', 'A15': 'Int64', 'A16': 'boolean'})

In [10]:
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,True,58.67,4.460,u,g,q,h,3.04,True,True,6,True,g,43,560,True
1,True,24.50,0.500,u,g,q,h,1.50,True,True,0,True,g,280,824,True
2,False,27.83,1.540,u,g,w,v,3.75,True,True,5,True,g,100,3,True
3,False,20.17,5.625,u,g,w,v,1.71,True,True,0,True,s,120,0,True
4,False,32.08,4.000,u,g,m,v,2.50,True,True,0,True,g,360,0,True


In [11]:
df.isnull().any(axis=1).sum()

37

In [12]:
#dane z nullami można wydropić bo jest ich mniej niż 5%, ale uzupełnimy medianą (bądź moda), bo to nadal dośc mały zbiór danych

In [13]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')

In [14]:
categorical = ['A4', 'A5', 'A6', 'A7', 'A13']

In [15]:
df_cat = df[categorical]
df_num = df.drop(categorical, axis=1)

In [16]:
imp_mean.fit(df_num)

SimpleImputer(strategy='median')

In [ ]:
def preprocess(df):
    df = df.replace('?', np.NaN)
    df = df.replace({'A1': {'a': 1, 'b': 0}, 'A9': {'t': 1, 'f': 1}, 'A10': {'t': 1, 'f': 1}, 'A12': {'t': 1, 'f': 1}, 'A16': {'+': 1, '-': 0}})
    df = df.astype({'A1': 'boolean', 'A2': 'float64', 'A3': 'float64', 'A8': 'float64', 'A9': 'boolean', 'A10': 'boolean', 'A11': 'Int64', 'A12': 'boolean', 'A14': 'Int64', 'A15': 'Int64', 'A16': 'boolean'})
def cross_validation(model):
    #split data
    #impude data
    #learn model
    pass
